# Final Function

### Load Packages

In [152]:
import pandas as pd
import numpy as np
import sys
import re
import io
from shapely import wkt
import geckodriver_autoinstaller
import folium
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from ipywidgets import interact,interact_manual,HBox,Output,Tab,widgets
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# Adapt accordingly https://pypi.org/project/webdriver-manager/
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager # Code 2
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

### Specify model to be used

In [185]:
# Load Model to be used
model_name = "PSPNet_sm_Aug-04-2022-1349"

from keras.models import load_model
model = load_model(f"models/{model_name}.hdf5", compile=False)

### Define necessary functions

In [184]:
# Get image file
def make_map(pt, ts, size=640):
    """
    create folium map with a given center, 
    optionally fit and show a geometry and/or a larger bounding box
    """
    #pt = (pt[1], pt[0])

    # Create satellite image map for the coordinate point specified
    m = folium.Map(location=pt, min_zoom=17, max_zoom=18, width=size, height=size, zoom_control=False, attribution_control=False)
    
    # First tile server
    if ts == 1:
        folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True
        ).add_to(m)
    
    # Second tile server
    elif ts == 2:
        folium.TileLayer(
            tiles = 'https://clarity.maptiles.arcgis.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True
        ).add_to(m)
        
    #m.fit_bounds(bounds, max_zoom = 18)
    #m.fit_bounds(crop_bounds) # FLAG
    
    img_data = m._to_png()
    img = Image.open(io.BytesIO(img_data))
    
    img_size = (0,0,size,size)
    img = img.crop(img_size)
    rgb_im = img.convert("RGB")
    image = np.array(rgb_im)
    img2 = scaler.fit_transform(image.reshape(-1, image.shape[-1])).reshape(image.shape)
    
    return img2

In [186]:
# Predict on a loaded image
def get_prediction(gps):
    test_img = make_map(gps, 2, 480)
    
    test_img_input = np.expand_dims(test_img, 0)

    prediction = (model.predict(test_img_input, verbose=0))
    predicted_img=np.argmax(prediction, axis=3)[0,:,:]
    #return predicted_img
    plt.figure(figsize=(10, 10))

    #plt.title('Prediction')
    plt.imshow(test_img)

    col_a = np.array([(0.1, 0.2, 0.5, 0), "yellow"], dtype=object)
    cmap_c = ListedColormap(col_a)

    plt.imshow(predicted_img, cmap=cmap_c, alpha=0.3)
    plt.axis('off')
    plt.show()

In [183]:
# Some parking lot locations for testing
gps_directory = {0: (-17.9495555, 25.8216612),
                 1: (-37.5922341, 144.2417904),
                 2: (-32.2087314, 148.6231326),
                 3: (51.6758745, 7.0868117),
                 4: (-24.4068105, 150.4983079),
                 5: (48.16320, 16.00519)}

In [ ]:
get_prediction(gps_directory[5])

### Final Interactive Function to get Predictions from coordinates

In [190]:
# Interactive Function for Predictions
print("Generate Prediction for Custom Coordinates")
interact_generate = interact_manual.options(manual_name="Get Prediction")

@interact_generate(latitude="48.18516", longitude="15.79659")

def inter_pred(latitude,longitude):
    gps = (float(latitude), float(longitude))
    return get_prediction(gps)

Generate Prediction for Custom Coordinates


interactive(children=(Text(value='48.18516', description='latitude'), Text(value='15.79659', description='long…